In [22]:
!pip install torch transformers -q

In [24]:
from transformers import BertTokenizer, BertForQuestionAnswering
import torch

# Load the BERT model and tokenizer
tokenizer = BertTokenizer.from_pretrained('deepset/bert-base-cased-squad2')
model = BertForQuestionAnswering.from_pretrained('deepset/bert-base-cased-squad2')



Some weights of the model checkpoint at deepset/bert-base-cased-squad2 were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [25]:
# Define the input text and question
text = "In a shocking turn of events, it was revealed that the once respected businessman had been embezzling funds from the company for years. Despite his deceit, many of his employees expressed their loyalty and support for him."
question = "Who was revealed to have been embezzling funds from the company?"

# Encode the input text and question, and get the scores for each word in the text
input = tokenizer(question, text,  return_tensors="pt")

output = model(**input)

# Find the words in the text that corresponds to the highest start and end scores
# with a torch.no_grad():
#     outputs = model(**inputs)
start_index = output.start_logits.argmax()
end_index = output.end_logits.argmax() + 1

# Extract the span of words as the answer
answer = tokenizer.decode(input.input_ids[0, start_index:end_index])
print(answer)


the once respected businessman


In [26]:
data = [("Hello, how can I help you today?", "Hi, I need some help with my order."),
        ("Sure, what seems to be the problem?", "I never received my order and it's been over a week."),
        ("I'm sorry to hear that. Let me check on that for you.", "Thank you. Can you also check on the status of my refund?"),
        ("Certainly. I will check on that as well.", "Thank you. Can you also provide me with the contact information for your supervisor?"),
        ("Of course. Here is the phone number and email address for our supervisor.", "Thank you for your help.")]

questions = [item[0] for item in data]
answers = [item[1] for item in data]

from transformers import BertTokenizer, BertForSequenceClassification
import torch
# Set the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load the BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

# Convert the data to input format for BERT
question_input_ids = []
question_attention_masks = []
for i in range(len(questions)):
    # Tokenize the question
    question_tokens = tokenizer.tokenize(questions[i])

    max_length = 512
    # Pad the input tokens
    question_tokens = question_tokens + [tokenizer.pad_token] * (max_length - len(question_tokens))

    # Create the input ids for the BERT model
    question_input_ids.append(tokenizer.convert_tokens_to_ids(question_tokens))

    # Create the attention masks for the input tokens
    question_attention_masks.append([1 if token != tokenizer.pad_token else 0 for token in question_tokens])

answer_input_ids = []
answer_attention_masks = []
for i in range(len(answers)):
    # Tokenize the answer
    answer_tokens = tokenizer.tokenize(answers[i])

    # Pad the input tokens
    answer_tokens = answer_tokens + [tokenizer.pad_token] * (max_length - len(answer_tokens))

    # Create the input ids for the BERT model
    answer_input_ids.append(tokenizer.convert_tokens_to_ids(answer_tokens))

    # Create the attention masks for the input tokens
    answer_attention_masks.append([1 if token != tokenizer.pad_token else 0 for token in answer_tokens])

# Concatenate the question and answer input lists
input_ids = question_input_ids + answer_input_ids
attention_masks = question_attention_masks + answer_attention_masks

# Convert the input ids and attention masks to tensors
input_ids = torch.tensor(input_ids).to(device)
attention_masks = torch.tensor(attention_masks).to(device)



In [28]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

In [29]:
# Define the criterion
criterion = torch.nn.CrossEntropyLoss()

def calculate_accuracy(predictions, labels):
    correct_predictions = 0
    for i in range(len(predictions)):
        if predictions[i] == labels[i]:
            correct_predictions += 1
    return correct_predictions / len(predictions)

# Set the number of epochs
num_epochs = 10

# Set the labels for the data
labels = [0 if i < len(questions) else 1 for i in range(len(questions) + len(answers))]
labels = torch.tensor(labels).to(device)

# Set the training loop
for epoch in range(num_epochs):

    # Set the training mode
    model.train()

    # Clear the gradients
    model.zero_grad()

    # Forward pass
    model = model.to(device)
    output = model(input_ids, attention_mask=attention_masks)

    # Calculate the loss
    loss = criterion(output[0], labels)

    # Backward pass
    loss.backward()

    # Update the parameters
    #torch.optimizer.OPTIMIZER.STEP()
    optimizer.step()

    # Print the loss and accuracy
    print("Epoch {}/{} - Loss: {:.5f} - Accuracy: {:.5f}".format(epoch + 1, num_epochs, loss.item(), calculate_accuracy(output[0].argmax(dim=1).cpu().numpy(), labels.cpu().numpy())))


Epoch 1/10 - Loss: 5.64962 - Accuracy: 0.00000
Epoch 2/10 - Loss: 2.46114 - Accuracy: 0.60000
Epoch 3/10 - Loss: 1.73734 - Accuracy: 0.80000
Epoch 4/10 - Loss: 2.57871 - Accuracy: 0.50000
Epoch 5/10 - Loss: 3.23318 - Accuracy: 0.40000
Epoch 6/10 - Loss: 1.97133 - Accuracy: 0.50000
Epoch 7/10 - Loss: 0.79754 - Accuracy: 0.80000
Epoch 8/10 - Loss: 0.29000 - Accuracy: 0.80000
Epoch 9/10 - Loss: 0.25680 - Accuracy: 1.00000
Epoch 10/10 - Loss: 0.28785 - Accuracy: 0.90000


In [31]:
# Set the model to eval mode
model.eval()

# Define the input
input_text = "Hello, how can I help you today?"

# Tokenize the input
input_tokens = tokenizer.tokenize(input_text)

# Pad the input tokens
input_tokens = input_tokens + [tokenizer.pad_token] * (max_length - len(input_tokens))

# Convert the input tokens to input ids
input_ids = tokenizer.convert_tokens_to_ids(input_tokens)

# Create the attention mask for the input
attention_mask = [1 if token != tokenizer.pad_token else 0 for token in input_tokens]

# Convert the input ids and attention mask to tensors
input_ids = torch.tensor(input_ids).unsqueeze(0).to(device)
attention_mask = torch.tensor(attention_mask).unsqueeze(0).to(device)

# Get the model output
output = model(input_ids, attention_mask=attention_mask)

# Get the predicted label
prediction = output[0].argmax(dim=1).item()

# Print the output
if prediction == 0:
    print("Question: {}".format(input_text))
else:
    print("Answer: {}".format(answers[prediction - 1]))



In [ ]:
!pip install streamlit


In [51]:
import streamlit as st

In [51]:
%%writefile app.py
import streamlit as st
import torch

from transformers import BertTokenizer, BertForQuestionAnswering, BertForSequenceClassification
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
max_length = 512
model = BertForQuestionAnswering.from_pretrained('deepset/bert-base-cased-squad2')
model = model.to(device)
data = [("Hello, how can I help you today?", "Hi, I need some help with my order."),
        ("Sure, what seems to be the problem?", "I never received my order and it's been over a week."),
        ("I'm sorry to hear that. Let me check on that for you.", "Thank you. Can you also check on the status of my refund?"),
        ("Certainly. I will check on that as well.", "Thank you. Can you also provide me with the contact information for your supervisor?"),
        ("Of course. Here is the phone number and email address for our supervisor.", "Thank you for your help.")]

questions = [item[0] for item in data]
answers = [item[1] for item in data]
print(answers)
def get_answer(input_text):
  # Define the input
  input_text = input_text

  # Tokenize the input
  input_tokens = tokenizer.tokenize(input_text)

  # Pad the input tokens
  input_tokens = input_tokens + [tokenizer.pad_token] * (max_length - len(input_tokens))

  # Convert the input tokens to input ids
  input_ids = tokenizer.convert_tokens_to_ids(input_tokens)

  # Create the attention mask for the input
  attention_mask = [1 if token != tokenizer.pad_token else 0 for token in input_tokens]

  # Convert the input ids and attention mask to tensors
  input_ids = torch.tensor(input_ids).unsqueeze(0).to(device)
  attention_mask = torch.tensor(attention_mask).unsqueeze(0).to(device)

  # Get the model output
  output = model(input_ids, attention_mask=attention_mask)

  # Get the predicted label
  prediction = output[0].argmax(dim=1).item()

  # Print the output
  if prediction == 0:
      return ("Question: {}".format(input_text))
  else:
      return ("Answer: {}".format(answers[prediction - 1]))



def generate_output(input_text):
    # Modify this function to generate the desired output based on the input text.
    # For demonstration purposes, let's just return the input text as output.
    return get_answer(input_text)

def main():
    st.title("Text Generator")

    # Text input from the user
    input_text = st.text_area("Enter some text:", "")

    if st.button("Generate"):
        if input_text:
            # Call the function to generate the output
            output_text = generate_output(input_text)
            st.write("Generated Output:")
            st.write(output_text)
        else:
            st.warning("Please enter some text!")

if __name__ == "__main__":
    main()


Overwriting app.py


In [52]:
!wget -q -O - ipv4.icanhazip.com
!streamlit run app.py & npx localtunnel --port 8501

35.223.42.208
[..................] / rollbackFailedOptional: verb npm-session 0e99bc09fd7791a


  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.12:8501
  External URL: http://35.223.42.208:8501

npx: installed 22 in 2.352s
your url is: https://polite-clocks-yell.loca.lt
Some weights of the model checkpoint at deepset/bert-base-cased-squad2 were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
['Hi, I need some help 